In [1]:
import sys
sys.path.append('..') #to add top-level to path

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.api import OLS
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.metrics import r2_score, accuracy_score

from pandas.plotting import scatter_matrix
#import seaborn as sns

import warnings
warnings.filterwarnings('ignore')


from modules.project_helper import VolFeatures, FuturesCloseData

In [2]:
import pickle
file = open("../data/features/full_features.pkl",'rb')
full_features = pickle.load(file)

In [121]:
instrument_list = ['ES', 'NQ', 'CD', 'EC', 'JY', 'MP', 'TY', 'US', 'C', 'S', 'W', 'CL', 'GC']
x_dict={}
y_dict={}
y_returns={}
for inst in instrument_list:
    #y_dict[inst] = 2 * (full_features[inst][inst] >=0)- 1
    y_dict[inst] = (full_features[inst][inst]>=0).astype(int)
    y_returns[inst] = full_features[inst][inst]
    x_dict[inst] = full_features[inst].drop([inst], axis=1)

In [147]:
class MLModel:
    def __init__(self,model,inst,x_dict,y_dict,y_returns,hyper_parameters={}):
        self.inst = inst
        self.x = x_dict[inst]
        self.y = y_dict[inst]
        self.y_returns = y_returns[inst]
        self.hyper_parameters=hyper_parameters

        self.model = model
        self.accuracy_train = None
        self.accuracy_test = None
        self.sharpe = None
    
        self.train_predictions = None
        self.test_predictions = None
        self.position = None
        
        self.train_class_balance = None
        self.test_class_balance = None
    
    def split_data(self):
        self.X_train, \
        self.X_test, \
        self.y_train, \
        self.y_test,\
        self.y_returns_train,\
        self.y_returns_test = train_test_split(self.x, self.y, self.y_returns, test_size=0.50, shuffle=False)


    def train_model(self):
        #self.model = OLS(self.y_train, self.X_train)
        #self.model = self.model.fit()

        self.model = self.model(random_state=0,**self.hyper_parameters).fit(self.X_train,self.y_train)

    def evaluate_sharpe(self, cutoff=0.55):
        rets = self.strategy_returns(cutoff)[0]
        self.sharpe = np.mean(rets) / np.std(rets)
    
    def get_position(self, cutoff=0.55):
        # converting predictions from {0,1} to {-1,1}, short/long
        self.position = 2 * self.model.predict(self.X_test) - 1
        self.position[self.model.predict_proba(self.X_test).max(axis=1) <= cutoff] = 0
        return self.position
    
    def strategy_returns(self,cutoff=0.55):
        x = self.get_position(cutoff=cutoff)[:-2]
        y = self.y_returns_test[:-2]
        strat_rets = x * y
        strat_rets_cum = (1 + strat_rets).cumprod()
        return strat_rets, strat_rets_cum


    def evaluate_model(self):
        self.accuracy_train = self.model.score(self.X_train, self.y_train)
        self.accuracy_test = self.model.score(self.X_test, self.y_test)
        self.evaluate_sharpe()
    
    def generate_predictions(self):
        self.train_predictions = self.model.predict(self.X_train)
        self.test_predictions = self.model.predict(self.X_test)
        
        self.train_class_balance = np.mean(self.train_predictions)
        self.test_class_balance = np.mean(self.test_predictions)
    



        
class AssetModels:
    def __init__(self,inst,x_dict,y_dict,y_returns,hyper_parameters):
        #self.model = None
        self.logistic_model = MLModel(LogisticRegression,inst,x_dict,y_dict,y_returns,
                                     hyper_parameters.get('logistic') if hyper_parameters.get('logistic') else {})
        self.rf_model = MLModel(RandomForestClassifier,inst,x_dict,y_dict,y_returns,
                                     hyper_parameters.get('rf') if hyper_parameters.get('rf') else {})
        self.tree_model = MLModel(DecisionTreeClassifier,inst,x_dict,y_dict,y_returns,
                                     hyper_parameters.get('tree') if hyper_parameters.get('tree') else {})
        self.boosted_tree_model = MLModel(GradientBoostingClassifier,inst,x_dict,y_dict,y_returns,
                                     hyper_parameters.get('boosted_tree') if hyper_parameters.get('boosted_tree') else {})

        self.ml_models = {'logistic':self.logistic_model,
                          'rf':self.rf_model,
                          'tree':self.tree_model,
                          'boosted_tree':self.boosted_tree_model
                         }
        
        self.best_model_name = None
        self.best_model = None
        
        self.best_model_accuracy = None
        self.best_model_sharpe = None

        self.accuracies_train = None
        self.accuracies_test = None
        self.sharpe_values = None
        
    def get_best_model(self):
        self.accuracies_test = pd.DataFrame.from_dict(
            {model_name:model.accuracy_test for model_name,model in self.ml_models.items()},
            orient='index',columns=['test_accuracy']
        )
        self.accuracies_train = pd.DataFrame.from_dict(
            {model_name:model.accuracy_train for model_name,model in self.ml_models.items()},
            orient='index',columns=['test_accuracy']
        )
        
        self.sharpe_values = pd.DataFrame.from_dict(
            {model_name:model.sharpe for model_name,model in self.ml_models.items()},
            orient='index',columns=['sharpe']
        )

        
        self.best_model_name = self.accuracies_test.idxmax().tolist()[0]
        self.best_model = self.ml_models.get(self.best_model_name)
        
        self.best_model_accuracy = self.accuracies_test[self.accuracies_test.index==self.best_model_name]
        self.best_model_accuracy.index.name = 'best_model'
        self.best_model_accuracy.reset_index(inplace=True)
        
        self.best_model_sharpe = self.sharpe_values[self.sharpe_values.index==self.best_model_name]
        #self.best_model_sharpe.index.name = 'best_model'
        #self.best_model_sharpe.reset_index(inplace=True)

        
        
    def run(self):
        {model.split_data() for model in self.ml_models.values()}
        {model.train_model() for model in self.ml_models.values()}
        {model.evaluate_model() for model in self.ml_models.values()}
        {model.generate_predictions() for model in self.ml_models.values()}
        self.get_best_model()

        
class ModelBuildier:
    def __init__(self,x_dict,y_dict,y_returns,instrument_list,hyper_parameters={}):
        self.x_dict = x_dict
        self.y_dict = y_dict
        self.hyper_parameters = hyper_parameters
        self.instrument_list = instrument_list
        self.asset_models =  {inst: AssetModels(inst,x_dict,y_dict,y_returns,
                            hyper_parameters.get(inst) if hyper_parameters.get(inst) else {})\
                              for inst in instrument_list}
        
        self.accuracies = pd.DataFrame()
        
    def get_accuracies(self):
        for inst in instrument_list:
            accuracy_df = self.asset_models[inst].best_model_accuracy
            accuracy_df.index = [inst]
            sharpe_df = self.asset_models[inst].best_model_sharpe
            sharpe_df.index = [inst]
            accuracy_df = accuracy_df.join(sharpe_df)
            self.accuracies = self.accuracies.append(accuracy_df)
        
    
    def run(self):
        {inst: model.run() for inst,model in self.asset_models.items()}
        self.get_accuracies()
        
        

In [148]:
hp = {
    'ES':{
        'logistic':{
            'C':0.1
        }
    },

    'NQ':{
        'logistic':{
            'C':1
        }
    },
    'CD':{
        'logistic':{
            'C':0.1
        }
    },
    'EC':{
        'logistic':{
            'C':100000
        }
    },
    'JY':{
        'logistic':{
            'C':100000
        }
    },
    'MP':{
        'logistic':{
            'C':0.1
        }
    },
    'TY':{
        'logistic':{
            'C':0.1
        }
    },
        
    'US':{
        'logistic':{
            'C':0.1
        }
    },
    'C':{
        'logistic':{
            'C':1
        }
    },    
    'S':{
        'logistic':{
            'C':1000
        }
    },
    'W':{
        'logistic':{
            'C':10
        }
    },
    'CL':{
        'logistic':{
            'C':0.01
        }
    },
    'GC':{
        'logistic':{
            'C':0.1
        }
    }




}


In [149]:
model_builder = ModelBuildier(x_dict,y_dict,y_returns,instrument_list,hyper_parameters=hp)
model_builder.run()

In [150]:
model_builder.accuracies

,best_model,test_accuracy,sharpe
ES,logistic,0.622857,0.310880
NQ,logistic,0.640000,0.303768
CD,logistic,0.565714,0.161168
EC,tree,0.548571,0.087287
JY,boosted_tree,0.542857,0.088200
MP,logistic,0.568571,0.202234
TY,logistic,0.571429,0.121249
US,logistic,0.582857,0.115957
C,logistic,0.594286,0.176896
S,logistic,0.588571,0.186440


In [8]:
# imp_df = pd.DataFrame()
# for inst in instrument_list:
#     feature_imp_df = pd.DataFrame(models[inst].models['tree'].model.feature_importances_,columns=[inst])
#     feature_imp_df.index = x_dict[inst].columns
#     imp_df = imp_df.join(feature_imp_df,how='outer')

In [209]:
# imp_df.to_csv('imp_df.csv')